In [1]:
import pandas


# Descripción del Set de Datos

Hemos trabajo con un set de datos extraídos de Twitter con Tecnología de TI, donde se haya mencionado las siguientes tecnologías **CRM, Big Data y Base de datos (Databases)**.

La procedencia de los **Tweets es a nivel mundial**.

El objetivo es poder analizar el contenido de los textos alrededor de estas temáticas e identificar tendencias por país y contexto en el que se mencionan.

Por ejemplo que se dice alrededor de la palabra **CRM**, en que países, cuales son las tecnologías, marcas, de que país son los usuarios más influyentes.

Otra información a analizar serían los temas mencionados usando hash tags.



In [2]:
dataset = pandas.read_csv('datoskunan_ok.csv', sep=';', 
    encoding='utf-8')

In [3]:
dataset.describe()

,id,user_followers_count,retweet_count,favorite_count,retweeted_status_retweet_count,retweeted_status_id,retweeted_status_favorite_count,coordinates_coordinates_0,coordinates_coordinates_1,id_area
count,2.000000e+03,2.000000e+03,2000.0,2000.0,554.000000,5.540000e+02,554.000000,385.000000,385.000000,0.0
mean,8.652608e+17,4.063709e+03,0.0,0.0,23.646209,8.766125e+17,30.615523,92.705284,25.964457,NaN
std,1.614274e+16,4.240003e+04,0.0,0.0,74.897430,1.264598e+16,203.227285,24.523932,5.057165,NaN
min,8.403621e+17,0.000000e+00,0.0,0.0,1.000000,8.197759e+17,0.000000,-59.466667,-34.250000,NaN
25%,8.466830e+17,1.500000e+02,0.0,0.0,3.000000,8.805437e+17,3.000000,90.375000,23.700000,NaN
50%,8.703127e+17,7.395000e+02,0.0,0.0,9.000000,8.811653e+17,8.000000,90.375000,23.700000,NaN
75%,8.812459e+17,2.645000e+03,0.0,0.0,21.000000,8.817638e+17,21.000000,116.407395,31.230416,NaN
max,8.826464e+17,1.175835e+06,0.0,0.0,1358.000000,8.822071e+17,4056.000000,121.496494,40.046271,NaN


In [4]:
dataset.dtypes

id                                   int64
created_at                          object
user_name                           object
user_screen_name                    object
user_followers_count                 int64
text                                object
retweet_count                        int64
favorite_count                       int64
retweeted_status_retweet_count     float64
retweeted_status_id                float64
retweeted_status_favorite_count    float64
retweeted_status_text               object
entities_hashtags_0_text            object
entities_hashtags_1_text            object
entities_hashtags_2_text            object
entities_hashtags_3_text            object
lang                                object
user_location                       object
coordinates_coordinates_0          float64
coordinates_coordinates_1          float64
place_country                       object
place_country_code                  object
place_full_name                     object
place_id   

In [5]:
dataset['entities_hashtags_0_text']

0                        ASM
1                        NaN
2                     AMBA17
3                        NaN
4                    bigdata
5                        NaN
6                     oracle
7                        NaN
8                       coyb
9                        NaN
10                       NaN
11                       NaN
12                       NaN
13                      IaaS
14                       NaN
15                       NaN
16                   bigdata
17                       NaN
18                       NaN
19                    vettel
20                       NaN
21               spacetweeps
22                       NaN
23                SaReGaMaPa
24                       NaN
25                  FCBayern
26                   UCLdraw
27                       NaN
28                       NaN
29          TNTHulingTapatan
                ...         
1970                     IoT
1971                     NaN
1972                database
1973     Predi

Explorar soluciones con diferentes parámetros y compararlas. Por ejemplo, variar el número de clusters, las
métricas de distancia, el número de iteraciones o el número de veces que se inicializan las semillas. Describir
brevemente: número de clusters, población de cada cluster, algunas caracterı́sticas distintivas de cada cluster,
algunos elementos que se puedan encontrar en cada cluster.

In [6]:
from sklearn.datasets import load_files 

import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

from __future__ import print_function

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals import joblib
from sklearn.cluster import KMeans

In [7]:
# preparar funciones de procesamiento de texto
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [8]:
text = dataset.text.values

**Separar en palabras y usar las raíces de los vocablos**

In [9]:
palabras = []

for i in text:
    p = tokenize_and_stem(i)
    palabras.extend(p)

**Limpiar las "palabras vacias"**

In [10]:
# lista de "stopwords"
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('https')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
f_text = [word for word in palabras if word not in stopwords]
print(f_text)

['asm', 'tour', 'around', 'kingdom', 'mainten', 'contract', 'flour', 'mill', 'region', 'check', 'plc…', '//t.co/86i0lev9kv', 'float', 'oracl', 'middl', 'east', '//t.co/r3qa4inead', 'amba17', 'k2partner', 'salesforceu', 'great', 'event', '//t.co/mf2fkaadrz', 'right', 'cousin', '//t.co/nomvojgo7x', 'bigdata', 'onli', 'buzzword', 'bigdata', 'data', 'arabnetdubai', 'arabnet', 'arabnetm', 'arabnetm', '//t.co/zeisbigjkt', 'lucki', '//t.co/svned3ci0u', 'nearbi', 'love', 'old', 'hous', 'oracl', 'oracl', 'dubai', '//t.co/gtszgnw36l', 'claaiireeyyy', 'jm_galario', 'anneyeong', 'bessi', '//t.co/cg6nfdmdir', 'solid', 'blue', 'poor', 'perform', 'baggi', 'coyb', 'efc', 'danni', 'devito', 'treasur', '//t.co/ang3hb2akc', 'acsabudhabi', 'student', 'stand', 'human', 'traffick', 'amp', 'modern-day', 'slaveri', 'w/', 'power', 'perform', 'on…', '//t.co/kp5roczag', 'saw', 'iliad', 'doubl', 'tripl', 'wow', 'amaz', 'perform', '//t.co/fywjlmxfgk', 'get', 'good', 'respons', 'data', 'alway', 'good', 'pleas', 'pa

**Crear una tabla de palabras**

In [12]:
vocab_frame = pd.DataFrame({'words': f_text}, index = range(len(f_text)))
print(vocab_frame[0:10])

      words
0       asm
1      tour
2    around
3   kingdom
4   mainten
5  contract
6     flour
7      mill
8    region
9     check


In [13]:
# Compute the term frequency-inverse document frequency matrix
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, max_features=200000,
                                min_df=20, stop_words=stopwords,
                                use_idf=True, tokenizer=tokenize_and_stem, 
                                ngram_range=(1,5),
                                binary=True)

tfidf_matrix = tfidf_vectorizer.fit_transform(text)
print("La matrix tiene %i filas (documentos) y %i columnas (palabras)\n" % tfidf_matrix.shape)



La matrix tiene 2000 filas (documentos) y 245 columnas (palabras)



In [14]:
terms = tfidf_vectorizer.get_feature_names()
print("Hay en total %i palabras:\n" % len(terms))
print(terms)

Hay en total 245 palabras:

["'s", 'account', 'ai', 'ai machinelearn', 'amp', 'analyt', 'analyt innovation…', 'analyt leadership', 'analyt manag', 'analyt manag shanghai', 'analytics…', 'anyon', 'anyon job', 'app', 'appli', 'audit', 'audit analyt', 'audit analyt manag', 'audit analyt manag shanghai', 'b3', 'b3 disrupt', 'b3 disrupt b3', 'best', 'best open', 'best open sourc', 'big', 'big data', 'bigdata', 'bigdata analyt', 'bigdata datasci', 'bigdata iot', 'bigdata…', 'blockchain', 'blog', 'busi', 'carpet', 'carpet manufacturer/…', 'ceo', 'chang', 'clear', 'clear disrupt', 'clear disrupt b3', 'clear disrupt b3 disrupt', 'clear disrupt b3 disrupt b3', 'click', 'click appli', 'cloud', 'cmo', 'compani', 'crm', 'crt/crx/cb/crm/sacking/hessian', 'crt/crx/cb/crm/sacking/hessian jute', 'crt/crx/cb/crm/sacking/hessian jute yarn', 'crt/crx/cb/crm/sacking/hessian jute yarn world', 'crt/crx/cb/crm/sacking/hessian jute yarn world wide', 'custom', 'cybersecur', 'data', 'data analyt', 'databas', 'da

## Buscando grupos de documentos con K-Means


In [15]:
num_clusters = 10
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

# print(clusters)

In [16]:
# Recuento del número de elementos en cada cluster
for i in range(num_clusters):
    print ('El cluster %i tiene %i elementos' % (i, clusters.count(i)))

El cluster 0 tiene 27 elementos
El cluster 1 tiene 386 elementos
El cluster 2 tiene 244 elementos
El cluster 3 tiene 167 elementos
El cluster 4 tiene 645 elementos
El cluster 5 tiene 124 elementos
El cluster 6 tiene 117 elementos
El cluster 7 tiene 98 elementos
El cluster 8 tiene 100 elementos
El cluster 9 tiene 92 elementos


In [17]:
dist = 1 - cosine_similarity(tfidf_matrix)

In [18]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1]     
        
for i in range(num_clusters):
    print("[[ Cluster %d ]]" % i, end='\n\n')
    
    print("   WORDS /// ", end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(terms[ind], end=' / ')
    print('\n\n')


Top terms per cluster:

[[ Cluster 0 ]]

   WORDS /// job open / latest job / latest job open / open / hire read latest job / hire read latest / 


[[ Cluster 1 ]]

   WORDS /// bigdata / rt / iot / ai / datasci / machinelearn / 


[[ Cluster 2 ]]

   WORDS /// perform / 's / amp / today / great / dubai / 


[[ Cluster 3 ]]

   WORDS /// carpet manufacturer/… / manufacturer/… / jute yarn world wide / jute yarn world wide carpet / wide / wide carpet / 


[[ Cluster 4 ]]

   WORDS /// analyt / rt / amp / data / 's / crm / 


[[ Cluster 5 ]]

   WORDS /// oracl / cloud / 's / rt / amp / india / 


[[ Cluster 6 ]]

   WORDS /// databas / sourc / project / best open / best open sourc / open sourc / 


[[ Cluster 7 ]]

   WORDS /// job / click appli / shanghai / click / hire / appli / 


[[ Cluster 8 ]]

   WORDS /// salesforc / rt / salesforceblog / rt salesforceblog / blog / crm / 


[[ Cluster 9 ]]

   WORDS /// entrepreneur / director / saa / datascientist / cmo / sap / 




### Ahora lo limpiamos un poco más: STOPWORDS, STEMMING & TOKENIZING

In [19]:
dataset['clusters'] = clusters 
# frame = pd.DataFrame(dataset, index = clusters , columns = ['text', 'cluster','place_country'])

In [20]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1]     
        
for i in range(num_clusters):
    print("[[ Cluster %d ]]" % i)
    print ('El cluster %i tiene %i elementos' % (i, clusters.count(i)), end='\n\n')
    print("  WORDS /// ", end='')
    
    for ind in order_centroids[i, :3]: #replace 6 with n words per cluster
        #print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=' / ')
        print(' %s' % terms[ind], end=' /')
    print('\n')
    
    print("  Country: /// ", end='')
#     for contenido in frame.loc[i]['place_country'].values:
#         print(' %s / ' % contenido, end='')
#     print('\n\n')
    print (dataset[dataset['clusters']==i].groupby(['place_country']).count().sort_values(['id'], ascending=False)[:]['id'])
    print('\n')
    print (dataset[dataset['clusters']==i].groupby(['place_country','user_name']).count().sort_values(['id'], ascending=False)[:]['id'])
    print('\n')
    print (dataset[dataset['clusters']==i].groupby(['entities_hashtags_0_text']).count().sort_values(['id'], ascending=False)[:]['id'])
    print('\n')
    
# entities_hashtags_0_text            object
# entities_hashtags_1_text            object
# entities_hashtags_2_text            object
# entities_hashtags_3_text            object

print('\n')

Top terms per cluster:

[[ Cluster 0 ]]
El cluster 0 tiene 27 elementos

  WORDS ///  job open / latest job / latest job open /

  Country: /// place_country
People's Republic of China    23
Bangladesh                     1
Name: id, dtype: int64


place_country               user_name                       
People's Republic of China  9dedaa716137e68b13461ab1673fe8bc    7
                            6d88accca0cace0a65e6212a71bbb607    6
                            63e69048c5ade464c8022dc1137a9df1    5
                            a5f1b8c67aaaa80d32c316255fc2ab3d    4
Bangladesh                  a5f1b8c67aaaa80d32c316255fc2ab3d    1
People's Republic of China  a023e30e5e1d468d375df1e7a95164c5    1
Name: id, dtype: int64


entities_hashtags_0_text
hiring    23
job        2
Name: id, dtype: int64


[[ Cluster 1 ]]
El cluster 1 tiene 386 elementos

  WORDS ///  bigdata / rt / iot /

  Country: /// place_country
United Arab Emirates          26
Canada                         5
People's Repu

Name: id, dtype: int64


entities_hashtags_0_text
Oracle                   18
India                     6
cloud                     5
ORACLE                    4
BusinessAnalytics         4
oracle                    4
hnytwtr                   3
IaaS                      3
Cloud                     2
ERP                       2
meoug                     2
C17LV                     2
startups                  2
BizQuote                  2
big                       1
fail                      1
finance                   1
Analytics                 1
iphone7plus               1
astrology                 1
oracledigital             1
social                    1
spring                    1
balance                   1
SME                       1
VAT                       1
SuiteWorld17              1
Startups                  1
SWS17                     1
RoyalNavalDockyard        1
PaaS                      1
OracleSMBDigiSummit       1
NBAFinals                 1
HCMWorld                  

Hemos encontrado mediante el uso de extracción de palabras y clustering los tópicos que necesitabamos ver, por ejemplo:

En caso del Cluster donde se menciona **CRM** el top 5 en los países está liderado por:

United Arab Emirates          128
Canada                         37
People's Republic of China     29
Australia                      20
Switzerland                    14

Los usuarios más influyentes son de origen:
place_country               user_name                       
United Arab Emirates        c5ee23e54567836b4b014da43c51f46c    7
                            4b4c5bba861617aef77f6e739369d972    6
Switzerland                 9c46b9290f4bd7ccfc7113a67ce219e9    5
Canada                      81abcea6e16bc538d9843e8808b2066b    5
Switzerland                 4956b848ab3c9cad4273461fa22221e5    4
People's Republic of China  6d88accca0cace0a65e6212a71bbb607    3

Y los hashtags más usados son:

CRM                    11
python                 10
OpenData                8
startups                7
BigData                 7
Blockchain              5
mysql                   4
BI                      4
BTSBBMAs                4
AI                      3
dataviz                 3
Hadoop                  3
MachineLearning         3
